# 1. Проблема

![task3](task3.jpeg)

# 2. Содержательная постановка

Необходимо определить процентный состав шихты для выплавки стали, нормированной по ГОСТу и обладающей минимальной стоимостью материалов с учетом ограничений на количество материалов.

# 3. Формальная постановка

Пусть $x_1,x_2,x_3$ - процентная доля компонентов шихты для выплавки стали.

Задача:

$$
(C,x)\rightarrow \min_{x_1+x_2+x_3 = 200}
$$

Ограничения нормированности по ГОСТу (условия):

$
0.15x_1 + 0.15x_2 + 0.17x_3 \geq 0.16 \times 200
$

$
0.15x_1 + 0.15x_2 + 0.17x_3 \leq 0.18 \times 200
$

$
0.10x_1 + 0.08x_2 + 0.09x_3 \leq 0.09 \times 200
$

$
0.7x_1 + 0.3x_2 + 0.5x_3 \leq 0.5 \times 200
$

$
x \geq 0
$

Ограничения на количество материалов:

$
x_1 \leq 150\\
x_2 \leq 100\\
x_3 \leq 75
$

# 4. Алгоритм и ПО

В качестве ПО будем использовать ЯП python, с подключенными модулями:

- numpy - для работы с линейной алгеброй

- cvxpy - для работы с целочисленным линейным программированием

# 5. Решение задачи

Приведем решение задачи с применением выбранного алгоритма.

Импортируем библиотеки

In [1]:
import cvxpy
import numpy as np

Внесем данные задачи и объявим переменную $x$

In [105]:
c = np.array([300,200,150])

A = np.array([[0.15,0.1,0.7],
              [0.15,0.08,0.3],
              [0.17,0.09,0.5]]).T

b = np.array([0.16, 0.18, 0.09, 0.5]) * 200

x = cvxpy.Variable(shape=3, integer = False)

Запишем условия задачи оптимизации и значение, которое будем минимизировать

In [123]:
constraints = [(A[0,:] @ x >= b[0]),
               (A[0,:] @ x <= b[1]),
               (A[1,:] @ x <= b[2]),
               (A[2,:] @ x <= b[3]),
               (sum(x) == 200),
               (x >= 0),
               (x[0] <= 150),
               (x[1] <= 100),
               (x[2] <= 75)]
               
total_value = c.T @ x

Решим задачу минимизации и получим ответ:

In [131]:
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=constraints)
print('Стоимость: %s' %problem.solve())
try:
    print('Доли материалов: {} : {} : {}'.format(x.value[0],x.value[1],x.value[2]))
except TypeError:
    print('В рамках данных условий задача не решаема')

Стоимость: inf
В рамках данных условий задача не решаема


# 6. Анализ

В ходе решения этой задачи был получен ожидаемый ответ: нет решения.

Изначально было видно, что решения, подходящего нашим условиям нет, т.к. у нас никогда не выполнится условие по процентному содержанию хрома. Необходимо либо подкорректировать состав, либо найти дополнительных поставщиков.